# Vérification de l'intégrité de la clé LDA et transformation de la base des stations (jointe avec celle de l'Insee)
Ce NoteBook a pour but de vérifier l'intégriter de la clé ZdC, seule code disponible pour faire une jointure avec les données de validation d'Île De France Mobilités (_IDFM_).

Il s'agit donc de vérifier que l'on peut faire la jointure sans problème (i.e. sans redoncance), puis de modifier la base si besoin.

## Packages et importation des données

In [1]:
import pandas as pd
from donnees_IDFM import donnees_validation_23
from pre_proces_Insee_IDFM import data_INSEE_stations

In [2]:
stations_RER=data_INSEE_stations()

ERROR 1: PROJ: proj_create_from_database: Open of /opt/mamba/share/proj failed


## Vérification de l'intégrité de la clé *ZdC* sur les données de *RER*

In [12]:
print(stations_RER['nom_ZdC'].nunique())
print(stations_RER['id_ref_ZdC'].nunique())

234
234


*A priori* il n'y a pas d'écart. Il reste à vérifier qu'il s'agit bien d'une petite erreur et non pas de mauvais matching.

### ZdC avec plusieurs noms de stations différents

In [4]:
stations_RER

,Geo Point,Geo Shape,gares_id,nom_long,nom_so_gar,nom_su_gar,id_ref_ZdC,nom_ZdC,id_ref_ZdA,nom_ZdA,...,ind_25_39,ind_40_54,ind_55_64,ind_65_79,ind_80p,ind_inc,pre_IDF,select,expt,
0,"48.53289010230884, 2.290082513840396","{""coordinates"": [2.290082513840396, 48.5328901...",101,Bouray,NaN,NaN,59830,Bouray,47905,Bouray,...,124.0,138.0,87.0,83.0,46.0,1.0,1.0,1.0,1.0,NaN
2,"48.529746917640566, 2.572073662735742","{""coordinates"": [2.572073662735742, 48.5297469...",85,Boissise-le-Roi,NaN,NaN,61943,Boissise-le-Roi,45763,Boissise-le-Roi,...,155.2,192.9,112.9,158.8,68.1,1.0,1.0,1.0,1.0,NaN
3,"48.725535303926115, 2.419565024387933","{""coordinates"": [2.419565024387933, 48.7255353...",3,Ablon,NaN,NaN,69554,Ablon,46387,Ablon,...,880.9,865.0,445.0,462.1,201.0,1.0,1.0,1.0,1.0,NaN
4,"48.932196005027805, 2.495513142584313","{""coordinates"": [2.495513142584313, 48.9321960...",37,Aulnay-sous-Bois,NaN,NaN,72646,Aulnay-sous-Bois,43071,Aulnay-sous-Bois,...,1205.5,1154.0,690.0,593.0,238.0,3.0,1.0,1.0,1.0,NaN
5,"48.79333629530144, 2.3222016336701206","{""coordinates"": [2.322201633670121, 48.7933362...",45,Bagneux,NaN,NaN,70210,Bagneux,44493,Bagneux,...,3069.0,2757.0,1675.0,1407.0,577.0,21.0,1.0,1.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,"48.869950134005364, 2.7822267234490363","{""coordinates"": [2.782226723449036, 48.8699501...",182,Marne-la-Vallée-Chessy,NaN,NaN,68385,Marne-la-Vallée Chessy,43239,Marne-la-Vallée Chessy,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,684025.2740
250,"48.962709512366416, 2.5126446278196126","{""coordinates"": [2.512644627819613, 48.9627095...",905,Villepinte,NaN,NaN,73547,Villepinte,58793,Villepinte,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,664313.9648
251,"49.00975645120457, 2.5612326952251054","{""coordinates"": [2.561232695225105, 49.0097564...",743,Aéroport Charles De Gaulle 1,NaN,NaN,73596,Aéroport CDG 1 (Terminal 3),462398,Aéroport CDG 1 (Terminal 3) - RER,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,667900.8037
252,"48.97342490628139, 2.5143602855334923","{""coordinates"": [2.514360285533492, 48.9734249...",621,Parc des Expositions,NaN,NaN,73568,Parc des Expositions,47878,Parc des Expositions,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,664446.9190


In [13]:
#On se restreint aux colonnes lda et LIBELLE_ARRET
sub_data = stations_RER[['id_ref_ZdC', 'nom_ZdC']]
#Suppression des doublons
unique_stations = sub_data.drop_duplicates()
#On aggrège et on filre pour avoir les stations qui ont structement plus de 1 LDA
multi_zdc_stations = unique_stations.groupby('nom_ZdC').filter(lambda x: x['id_ref_ZdC'].nunique() > 1)
#On trie par ordre alphabétique pour faciliter la lecture
multi_zdc_stations.sort_values(by='nom_ZdC')

,id_ref_ZdC,nom_ZdC


Pas de problème dans ce sens là.

### Noms de stations avec plusieurs ZdC

In [14]:
#On se restreint aux colonnes lda et LIBELLE_ARRET
sub_data = stations_RER[['id_ref_ZdC', 'nom_ZdC']]
#Suppression des doublons
unique_stations = sub_data.drop_duplicates()
#On aggrège et on filre pour avoir les stations qui ont structement plus de 1 LDA
multi_zdc_stations = unique_stations.groupby('id_ref_ZdC').filter(lambda x: x['nom_ZdC'].nunique() > 1)
#On trie par ordre alphabétique pour faciliter la lecture
multi_zdc_stations.sort_values(by='id_ref_ZdC')

,id_ref_ZdC,nom_ZdC


## Suppression des redondance des stations

La base que nous avions déjà préparées pour les stations de RER est donc quasi-opérationnelle. Il nous reste maintenant a éliminer les redondances car les stations avec correspondance sont comptées autant de fois qu'il y a de ligne (cela posera problème si l'on fait une jointure).

In [17]:
# Étape 1: Trouver les valeurs de 'id_ref_ZdC' qui apparaissent plus d'une fois.
duplicate_id_ref_ZdC = stations_RER['id_ref_ZdC'].value_counts()
duplicate_id_ref_ZdC = duplicate_id_ref_ZdC[duplicate_id_ref_ZdC > 1].index

# Étape 2: Filtrer le DataFrame pour afficher seulement les lignes avec les valeurs répétées de 'id_ref_ZdC'
stations_RER_duplicates = stations_RER[stations_RER['id_ref_ZdC'].isin(duplicate_id_ref_ZdC)]
stations_RER_duplicates

,Geo Point,Geo Shape,gares_id,nom_long,nom_so_gar,nom_su_gar,id_ref_ZdC,nom_ZdC,id_ref_ZdA,nom_ZdA,...,ind_25_39,ind_40_54,ind_55_64,ind_65_79,ind_80p,ind_inc,pre_IDF,select,expt,
39,"48.861822271863645, 2.3470126872387564","{""coordinates"": [2.347012687238756, 48.8618222...",163,Châtelet-Les Halles,NaN,NaN,474151,Châtelet les Halles,45102,Châtelet les Halles,...,8501.5,6004.0,2991.0,2856.0,863.0,20.0,1.0,1.0,1.0,NaN
40,"48.861822271863645, 2.3470126872387564","{""coordinates"": [2.347012687238756, 48.8618222...",164,Châtelet-Les Halles,NaN,NaN,474151,Châtelet les Halles,45102,Châtelet les Halles,...,8501.5,6004.0,2991.0,2856.0,863.0,20.0,1.0,1.0,1.0,NaN
41,"48.861822271863645, 2.3470126872387564","{""coordinates"": [2.347012687238756, 48.8618222...",170,Châtelet-Les Halles,NaN,NaN,474151,Châtelet les Halles,45102,Châtelet les Halles,...,8501.5,6004.0,2991.0,2856.0,863.0,20.0,1.0,1.0,1.0,NaN
57,"48.85353289978531, 2.34528833219743","{""coordinates"": [2.34528833219743, 48.85353289...",828,Saint-Michel,Notre-Dame,NaN,73620,Saint-Michel Notre-Dame,44877,Saint-Michel Notre-Dame,...,3148.1,2449.0,1398.0,1468.9,563.0,6.0,1.0,1.0,1.0,NaN
58,"48.852118881696434, 2.3440211520552796","{""coordinates"": [2.34402115205528, 48.85211888...",827,Saint-Michel,Notre-Dame,NaN,73620,Saint-Michel Notre-Dame,44877,Saint-Michel Notre-Dame,...,3148.1,2449.0,1398.0,1468.9,563.0,6.0,1.0,1.0,1.0,NaN
62,"48.8437725759169, 2.373916469806498","{""coordinates"": [2.373916469806498, 48.8437725...",307,Gare de Lyon,NaN,NaN,73626,Gare de Lyon,470195,Gare de Lyon,...,1894.4,1595.4,1062.1,864.0,266.0,28.0,1.0,1.0,1.0,NaN
63,"48.844139348918475, 2.373259976710645","{""coordinates"": [2.373259976710645, 48.8441393...",306,Gare de Lyon,NaN,NaN,73626,Gare de Lyon,470195,Gare de Lyon,...,1894.4,1595.4,1062.1,864.0,266.0,28.0,1.0,1.0,1.0,NaN
113,"48.881759488218286, 2.357570193470876","{""coordinates"": [2.357570193470876, 48.8817594...",309,Gare du Nord,NaN,NaN,478733,Gare du Nord,58572,Magenta,...,8465.5,5728.5,3116.0,2783.0,803.0,167.0,1.0,1.0,1.0,NaN
114,"48.881759488218286, 2.357570193470876","{""coordinates"": [2.357570193470876, 48.8817594...",310,Gare du Nord,NaN,NaN,478733,Gare du Nord,58572,Magenta,...,8465.5,5728.5,3116.0,2783.0,803.0,167.0,1.0,1.0,1.0,NaN
115,"48.88077520385858, 2.358732959087859","{""coordinates"": [2.358732959087859, 48.8807752...",490,Magenta,NaN,NaN,478733,Gare du Nord,58572,Magenta,...,8465.5,5728.5,3116.0,2783.0,803.0,167.0,1.0,1.0,1.0,NaN


Il y finallement 18 lignes pour 8 stations redondantes. On décide de completer la colonne *res_com* avec toutes les lignes en correspondance.

In [24]:
stations_RER.shape

(244, 64)

In [25]:
# Grouper les données par 'id_ref_ZdC' et appliquer une fonction personnalisée pour concaténer 'res_com'
def concatenate_res_com(group):
    # Concaténer toutes les 'res_com' avec un séparateur, par exemple une virgule ou un espace
    return ', '.join(group['res_com'])

# Appliquer la fonction personnalisée pour chaque groupe
aggregated_stations = stations_RER.groupby('id_ref_ZdC').apply(concatenate_res_com).reset_index(name='res_com')

In [26]:
aggregated_stations

,id_ref_ZdC,res_com
0,59420,RER D
1,59447,RER D
2,59470,RER D
3,59531,RER C
4,59565,RER D
...,...,...
229,478505,"RER D, RER C"
230,478733,"RER B, RER D, RER E"
231,478855,RER C
232,478926,RER A


In [27]:
unique_stations = stations_RER.drop_duplicates(subset='id_ref_ZdC').drop(columns=['res_com'])
final_stations = unique_stations.merge(aggregated_stations, on='id_ref_ZdC')

In [29]:
final_stations.shape

(234, 64)

Il y a bien 10 lignes de moins. On vérifie que l'opération a été faite correctement sur la station Gare du Nord.

In [31]:
final_stations[final_stations['nom_long']=='Gare du Nord'].loc[:,['nom_long', 'res_com']]

,nom_long,res_com
103,Gare du Nord,"RER B, RER D, RER E"


Pour une seule ligne on a bien accès à toutes les lignes. On vérifie qu'il n'y a plus de redondance pour être certain que la procédure a fonctionné correctement.

In [32]:
# Étape 1: Trouver les valeurs de 'id_ref_ZdC' qui apparaissent plus d'une fois.
duplicate_id_ref_ZdC = final_stations['id_ref_ZdC'].value_counts()
duplicate_id_ref_ZdC = duplicate_id_ref_ZdC[duplicate_id_ref_ZdC > 1].index

# Étape 2: Filtrer le DataFrame pour afficher seulement les lignes avec les valeurs répétées de 'id_ref_ZdC'
stations_RER_duplicates = final_stations[final_stations['id_ref_ZdC'].isin(duplicate_id_ref_ZdC)]
stations_RER_duplicates

,Geo Point,Geo Shape,gares_id,nom_long,nom_so_gar,nom_su_gar,id_ref_ZdC,nom_ZdC,id_ref_ZdA,nom_ZdA,...,ind_40_54,ind_55_64,ind_65_79,ind_80p,ind_inc,pre_IDF,select,expt,,res_com


In [34]:
final_stations.shape

(234, 64)

Tout est bon !